<a href="https://colab.research.google.com/github/daniely67/Credit-Scoring-Task/blob/main/CreditScoring_RuleBased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.read_json('user-wallet-transactions.json')

In [5]:
df.head()

,_id,userWallet,network,protocol,txHash,logId,timestamp,blockNumber,action,actionData,__v,createdAt,updatedAt
0,{'$oid': '681d38fed63812d4655f571a'},0x00000000001accfa9cef68cf5371a23025b6d4b6,polygon,aave_v2,0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...,0x695c69acf608fbf5d38e48ca5535e118cc213a89e3d6...,2021-08-17 05:29:26,1629178166,deposit,"{'type': 'Deposit', 'amount': '2000000000', 'a...",0,{'$date': '2025-05-08T23:06:39.465Z'},{'$date': '2025-05-08T23:06:39.465Z'}
1,{'$oid': '681aa70dd6df53021cc6f3c0'},0x000000000051d07a4fb3bd10121a343d85818da6,polygon,aave_v2,0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...,0xe6fc162c86b2928b0ba9b82bda672763665152b9de9d...,2021-05-20 15:36:53,1621525013,deposit,"{'type': 'Deposit', 'amount': '145000000000000...",0,{'$date': '2025-05-07T00:19:26.159Z'},{'$date': '2025-05-07T00:19:26.159Z'}
2,{'$oid': '681d04c2d63812d4654c733e'},0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...,0xe2d7eb815c89331a734ed6f204a06c385a1b39040baa...,2021-07-24 09:28:33,1627118913,deposit,"{'type': 'Deposit', 'amount': '100000000000000...",0,{'$date': '2025-05-08T19:23:47.877Z'},{'$date': '2025-05-08T19:23:47.877Z'}
3,{'$oid': '681d133bd63812d46551b6ef'},0x000000000096026fb41fc39f9875d164bd82e2dc,polygon,aave_v2,0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...,0x0d63a2eacd82b82f868db825ea7385e6bd8d046ee729...,2021-07-31 23:15:18,1627773318,deposit,"{'type': 'Deposit', 'amount': '400000000000000...",0,{'$date': '2025-05-08T20:25:33.141Z'},{'$date': '2025-05-08T20:25:33.141Z'}
4,{'$oid': '681899e4ba49fc91cf2f4454'},0x0000000000e189dd664b9ab08a33c4839953852c,polygon,aave_v2,0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...,0x590eabb812c5006a6f4766f44e6e9d3ad0b5b563de69...,2021-04-19 15:25:07,1618845907,redeemunderlying,"{'type': 'RedeemUnderlying', 'amount': '501548...",0,{'$date': '2025-05-05T10:58:45.934Z'},{'$date': '2025-05-05T10:58:45.934Z'}


In [6]:
# Convert amounts to float for calculations
def parse_amount(row):
    try:
        return float(row['actionData']['amount'])
    except:
        return np.nan

df['amount'] = df.apply(parse_amount, axis=1)

# Aggregate features
wallet_features = df.groupby('userWallet').agg(
    total_deposits=pd.NamedAgg(column='amount', aggfunc=lambda x: x[df['action'] == 'deposit'].sum()),
    num_deposits=pd.NamedAgg(column='action', aggfunc=lambda x: (x == 'deposit').sum()),
    num_withdrawals=pd.NamedAgg(column='action', aggfunc=lambda x: (x == 'redeemunderlying').sum())
).reset_index()

wallet_features['withdrawal_deposit_ratio'] = (
    wallet_features['num_withdrawals'] / wallet_features['num_deposits'].replace(0, np.nan)
).fillna(0)


In [7]:
def compute_score(row):
    score = 1000
    if row['withdrawal_deposit_ratio'] > 0.7:
        score -= 100
    if row['total_deposits'] == 0:
        score -= 400
    if row['num_deposits'] >= 3:
        score += 50
    score = max(0, min(score, 1000))
    return score

wallet_features['creditScore'] = wallet_features.apply(compute_score, axis=1)


In [8]:
wallet_features[['userWallet', 'creditScore']].to_csv('wallet_credit_scores.csv', index=False)


In [9]:
df1 = pd.read_csv('wallet_credit_scores.csv')

In [10]:
df1.head(20)

,userWallet,creditScore
0,0x00000000001accfa9cef68cf5371a23025b6d4b6,1000
1,0x000000000051d07a4fb3bd10121a343d85818da6,1000
2,0x000000000096026fb41fc39f9875d164bd82e2dc,1000
3,0x0000000000e189dd664b9ab08a33c4839953852c,600
4,0x0000000002032370b971dabd36d72f3e5a7bf1ee,1000
5,0x000000000a38444e0a6e37d3b630d7e855a7cb13,950
6,0x000000003853fcedcd0355fec98ca3192833f00b,1000
7,0x000000003ce0cf2c037493b1dc087204bd7f713e,1000
8,0x000000007858e6f2668e1e06111cfa24403a5466,1000
9,0x00000001a0f57e850c9db68b4a9bc34677437c5c,1000
